# package-lock.json to CSV converter

In [ ]:
#import packages
import pandas as pd
import requests
import warnings #warnings are annoying
warnings.filterwarnings('ignore')

In [ ]:
## load file into python
with open('./sample_package-lock.json') as package_lock_file:
    df = pd.read_json(package_lock_file) ##parse json into dataframe with pandas
    
df.head

In [ ]:
#change column names
df = df.rename(columns={'name':'Project','version':'Current Version','dependencies':'dict'})
df.index.name = "Packages"
#add a few empties
df['Latest Version'] = None
df['Integrity'] = None
df['Requires'] = None
df['Dependencies'] = None
df['hasDependencies'] = False
df.head

In [ ]:
# make and fill new columns
## since the dict column contains type dicts we can use the key/value pairs for easy data manipulation
for index,row in df.iterrows():
    df['Current Version'][index] = row['dict'].get('version')
    if row['dict'].get('integrity'):
        df['Integrity'][index] = row['dict'].get('integrity')
    if row['dict'].get('requires'):
        tempList = [key + ': ' +value for key,value in row['dict'].get('requires').items()].copy() ## turn dict into list
        df['Requires'][index] = tempList
        df['hasDependencies'][index] = True
    if row['dict'].get('dependencies'):
        tempList = [key + ': ' +row['dict'].get('dependencies')[key]['version'] for key in row['dict'].get('dependencies').keys()]
        df['Dependencies'][index] = tempList
        df['hasDependencies'][index] = True

df.head

In [ ]:
#drop columns
df.drop(labels={'dict','lockfileVersion'},axis=1,inplace=True)


df.head

In [ ]:
# in order to populate the 'Newest Version' column, we'll need to do some querys
## https://registry.npmjs.org/:package will suffice
### we'll also be making a LOT of requests
#### response.json() turns the response into a dict for us to parse

header = {'Accept' : 'application/vnd.npm.install-v1+json'} ##gets an abbreviated response

for index,rows in df.iterrows():
    response = requests.get('https://registry.npmjs.org/'+index,headers=header).json()
    df['Latest Version'][index] = response['dist-tags']['latest']

In [ ]:
##convert df to csv
df.to_csv('package-lock.csv')